In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision
import torchvision.transforms as transforms
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter(log_dir="logs") 

In [2]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
# Hyper-parameters
num_epochs = 4
batch_size = 10
learning_rate = 0.001

In [4]:
# 0) data import and set as pytorch dataset
# MNIST
train_dataset = torchvision.datasets.MNIST(root='./data', train=True,
    transform=transforms.ToTensor(), download=True)

test_dataset = torchvision.datasets.MNIST(root='./data', train=False,
    transform=transforms.ToTensor(), download=True)

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size,
    shuffle=True)

test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size,
    shuffle=True)

In [5]:
# 1) model build
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        # image shape is 1 * 28 * 28, where 1 is one color channel
        # 28 * 28 is the image size
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=3, kernel_size=5)    # output shape = 3 * 24 * 24
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)                       # output shape = 3 * 12 * 12
        # intput shape is 3 * 12 * 12
        self.conv2 = nn.Conv2d(in_channels=3, out_channels=9, kernel_size=5)    # output shape = 9 * 8 * 8
        # add another max pooling, output shape = 9 * 4 * 4
        self.fc1 = nn.Linear(9*4*4, 100)
        self.fc2 = nn.Linear(100, 50)
        # last fully connected layer output should be same as classes
        self.fc3 = nn.Linear(50, 10)

        
    def forward(self, x):
        # first conv
        x = self.pool(F.relu(self.conv1(x)))
        # second conv
        x = self.pool(F.relu(self.conv2(x)))
        # flatten all dimensions except batch
        x = torch.flatten(x, 1)
        # fully connected layers
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)

        return x


model = ConvNet().to(device)

In [6]:
# 2) loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [7]:
# 3) Training loop
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # init optimizer
        optimizer.zero_grad()
        
        # forward -> backward -> update
        outputs = model(images)
        loss = criterion(outputs, labels)
        writer.add_scalar("Loss/train", loss, epoch)
        loss.backward()

        optimizer.step()

        if (i + 1) % 1000 == 0:
            print(f'epoch {epoch+1}/{num_epochs}, step {i+1}/{n_total_steps}, loss = {loss.item():.4f}')
writer.close()
print('Finished Training')

epoch 1/4, step 1000/6000, loss = 2.2904
epoch 1/4, step 2000/6000, loss = 2.2858
epoch 1/4, step 3000/6000, loss = 2.3060
epoch 1/4, step 4000/6000, loss = 2.2829
epoch 1/4, step 5000/6000, loss = 2.2767
epoch 1/4, step 6000/6000, loss = 2.2753
epoch 2/4, step 1000/6000, loss = 2.2515
epoch 2/4, step 2000/6000, loss = 2.2708
epoch 2/4, step 3000/6000, loss = 2.2020
epoch 2/4, step 4000/6000, loss = 2.0236
epoch 2/4, step 5000/6000, loss = 1.3999
epoch 2/4, step 6000/6000, loss = 1.1021
epoch 3/4, step 1000/6000, loss = 0.3183
epoch 3/4, step 2000/6000, loss = 0.3344
epoch 3/4, step 3000/6000, loss = 0.5831
epoch 3/4, step 4000/6000, loss = 0.1724
epoch 3/4, step 5000/6000, loss = 0.1499
epoch 3/4, step 6000/6000, loss = 0.3646
epoch 4/4, step 1000/6000, loss = 0.5631
epoch 4/4, step 2000/6000, loss = 0.4260
epoch 4/4, step 3000/6000, loss = 0.3553
epoch 4/4, step 4000/6000, loss = 0.2882
epoch 4/4, step 5000/6000, loss = 0.7510
epoch 4/4, step 6000/6000, loss = 0.1272
Finished Trainin

In [8]:
_, predicted = torch.max(outputs, 1)
print(_, predicted)

tensor([ 7.0104,  7.3291,  7.6799,  7.3606,  5.4790,  5.7394,  8.2522,  6.9334,
        11.5940,  7.9955], grad_fn=<MaxBackward0>) tensor([1, 9, 5, 6, 0, 3, 3, 3, 4, 4])


In [9]:
print(len(outputs))

10


In [10]:
batch = next(iter(test_loader))
batch[1]

tensor([1, 1, 4, 3, 6, 8, 1, 8, 9, 5])

In [11]:
# 4) Testing loop
wrong = []
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        # max returns (value, index)
        _, predicted = torch.max(outputs, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()

        for i in range(batch_size):
            label = labels[i]
            pred = predicted[i]
            if (label == pred):
                n_class_correct[label] += 1
                n_class_samples[label] += 1
            else:
                wrong.append([pred, label])
               

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network: {acc} %')

    for i in range(10):
        acc = 100.0 * n_class_correct[i] / n_class_samples[i]
        print(f'Accuracy of {i}: {acc} %')

Accuracy of the network: 92.08 %
Accuracy of 0: 100.0 %
Accuracy of 1: 100.0 %
Accuracy of 2: 100.0 %
Accuracy of 3: 100.0 %
Accuracy of 4: 100.0 %
Accuracy of 5: 100.0 %
Accuracy of 6: 100.0 %
Accuracy of 7: 100.0 %
Accuracy of 8: 100.0 %
Accuracy of 9: 100.0 %
